# Analysis of National COVID-19 Virus Testing Data
This notebook reports the statistical analysis of all COVID-19 virus testing data from the US.  

In [ ]:
%load_ext rpy2.ipython
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%%R
## setting up R
system("add-apt-repository -y ppa:marutter/rrutter")
system("add-apt-repository -y ppa:marutter/c2d4u")
system("apt-get update")
system("apt install -y r-cran-rstan")
packages<-function(x, repos="http://cran.r-project.org", ...){
  x<-as.character(match.call()[[2]])
  if (!require(x,character.only=TRUE)){
    install.packages(pkgs=x, repos=repos, ...)
    require(x,character.only=TRUE)
  }
}

packages(tidyverse)
packages(reshape2)
packages(rv)
packages(lattice)
packages(rstan)

rstan_options(auto_write = TRUE)
options(mc.cores = min(c(parallel::detectCores(), 8)))

nchains <-  min(c(parallel::detectCores(), 8))
niters <- 5000000
nkeep <- 2500
nthin <- ceiling((niters/2)*nchains/nkeep)


## Data Source
Data used in this study are from [the COVID Tracking project](covidtracking.com).  Data from states are traacked and updated on [GitHub](github.com/COVID19Tracking/covid-tracking-data).  The daily data file includes all daily testing data, and the update file includes only the most recent data.  The program allows selecting data by date. 


In [22]:
%%R
covid_daily <- read.csv("https://raw.githubusercontent.com/COVID19Tracking/covid-tracking-data/master/data/states_daily_4pm_et.csv")
covid_update <- read.csv("https://raw.githubusercontent.com/COVID19Tracking/covid-tracking-data/master/data/states_current.csv")

covid_update<-covid_update %>% filter(!is.na(negative))
covid_update$n <- covid_update$negative+covid_update$positive
##print(covid_update)
covid_update$TimeCk <- as.Date(covid_update$checkTimeEt, format="%m/%d %H:%M")

covid_daily<-covid_daily %>% filter(!is.na(negative))
covid_daily$n <- covid_daily$negative+covid_daily$positive
##print(covid_update)
covid_daily$TimeCk <- as.Date(as.character(covid_daily$date), format="%Y%m%d")

print(covid_update)

   state positive negative pending death total lastUpdateEt checkTimeEt
1     AK        1      143      NA    NA   144   3/13 16:30  3/14 13:40
2     AL        6       22      46    NA    74   3/13 00:00  3/14 15:25
3     AR       12       65      26    NA   103   3/13 00:00  3/14 15:21
4     AZ       12      121      50     0   183   3/14 12:01  3/14 13:18
5     CA      252      916      NA     5  1168   3/13 11:00  3/14 15:48
6     CO      101      712      NA     1   814   3/14 14:00  3/14 15:42
7     CT       11      125      NA    NA   136   3/13 16:00  3/14 13:22
8     DC       10       49      10    NA    69   3/13 19:00  3/14 13:23
9     DE        6       36      32    NA    74   3/14 13:00  3/14 15:49
10    FL       77      478     221     3   776   3/14 00:54  3/14 13:25
11    IA       17       83      NA    NA   100   3/13 00:00  3/14 16:00
12    ID        2      174      NA    NA   176   3/13 22:45  3/14 16:16
13    IL       46      316      82    NA   444   3/13 00:00  3/1

## Statistical Model
A Bayesian hierarchical model on $\theta$:
$$
\begin{array}{rcl}
y_j &\sim& Bin(p_j, n_j)\\
p_j & = & \theta_j(1-f_n)+(1-\theta)f_p\\
\mathrm{logit}(\theta_j) & \sim & N(\mu_0, \sigma_0^2)\\
f_n & \sim & beta(\alpha_n, \beta_n)\\
f_p & \sim & beta(\alpha_p, \beta_p)
\end{array}
$$
where $j$ indexes states, $y_j$ is the number of positives, $n_j$ is the number of tests with results (excluding pending tests), $\theta_j$ is the prevalence for state $j$.  The hyper-parameter $\mu_0$ is the national average of prevalence (in logit scale) and $\sigma_0^2$ is among state variance of the logit transformed state-specific prevalences.

In [ ]:
%%R
 stan.input2 <- function(infile=covid_update, an=1, bn=99,
                        ap=1, bp=99, chains=nchains)
{
    date_checked <- format(unique(infile$TimeCk)[1], "%B_%d")
    y <- infile$positive
    n <- y+infile$negative
    gr <- infile$state
    N <- dim(infile)[1]
    inits <- list()
    bugs.data <- list(N=N, y=y, n=n, an=an, bn=bn, ap=ap, bp=bp)
    for (i in 1:chains)
        inits[[i]] <- list(logitth=rnorm(N, 0, 0.1), fn=runif(1, 0, 0.1),
                           fp=runif(1, 0, 0.1),
                           mu0=rnorm(1, 0, 0.1), sigma0=runif(1))
    para <- c("theta", "fn", "fp", "theta0", "sigma0")
    return(list(para=para, data=bugs.data, inits=inits,
                n.chains=chains, dateCkd=date_checked , model = "
           data{
            int N;
            int n[N];
            int y[N];
            real an;
            real bn;
            real ap;
            real bp;
           }
           parameters{
            real<lower=0,upper=1> fn;
            real<lower=0,upper=1> fp;
            vector[N] logitth;
            real mu0;
            real<lower=0> sigma0;
           }
           transformed parameters{
            vector[N] theta;
            for (i in 1:N){
            theta[i] = inv_logit(logitth[i]);
            }
           }
           model {
            fn ~ beta(an, bn);
            fp ~ beta(ap, bp);
            for (i in 1:N){
              logitth[i] ~ normal(mu0, sigma0);
             }
            y ~ binomial(n, theta*(1-fn)+(1-theta)*fp);
           }
           generated quantities{
             real<lower=0,upper=1> theta0;
             theta0 = inv_logit(mu0);
           }
           "))
}



To run the model using data from a specific date:

In [ ]:
tmp <- covid_daily$date==20200315
input.to.stan <- stan.input2(infile=covid_daily[tmp,])
date_checked <- input.to.stan$dateCkd

Use the most recent data:

In [ ]:

                
input.to.stan <- stan.input2()
date_checked <- input.to.stan$dateCkd

fit <- stan_model(model_code = input.to.stan$model)
fitB2 <- sampling(fit, data = input.to.stan$data, init=input.to.stan$inits,
                  pars=input.to.stan$para,
                  iter = niters, chains = input.to.stan$n.chains, thin=nthin,
                  control=list(adapt_delta=0.99))
print(fitB2)
save(fitB2, file=paste("covid19_fit2_",
                       input.to.stan$dateCkd,
                       ".RData", sep=""))

input.to.stan <- stan.input2(infile=covid_update, an=2, bn=22, ap=3, bp=23)
fitB3 <- sampling(fit, data = input.to.stan$data, init=input.to.stan$inits,
                  pars=input.to.stan$para,
                  iter = niters, chains = input.to.stan$n.chains, thin=nthin,
                  control=list(adapt_delta=0.99))
print(fitB3)
save(fitB3, file=paste("covid19_fit3_", input.to.stan$dateCkd,".RData", sep=""))

input.to.stan <- stan.input2(infile=covid_update,
                             an=16, bn=24, ap=4, bp=45)
fitB4 <- sampling(fit, data = input.to.stan$data, init=input.to.stan$inits,
                  pars=input.to.stan$para,
                  iter = niters, chains = input.to.stan$n.chains, thin=nthin,
                  control=list(adapt_delta=0.99))
print(fitB4)
save(fitB4, file=paste("covid19_fit4_",input.to.stan$dateCkd, ".RData", sep=""))


Processing output and producing prevalence plots

In [ ]:
## processing output

covid_plot <- function(stanfit=fitB2, rawdata=covid_daily[tmp,],
                       output_file1="prevalence1.png",
                       output_file2="prevalence2.png",
                       output_file3="prevalence3.png",
                       onscreen=T, chckd=date_checked){
    nn <- dim(rawdata)[1]
    covid_stan2 <- rvsims(as.matrix(as.data.frame(rstan::extract(stanfit,
                                                                 permute=T))))
    covid_summary2 <- as.data.frame(summary(covid_stan2))
    names(covid_summary2) <- c("name", "Mean", "sd", "X1","X2.5",
                               "X25","X50","X75","X97.5","X99", "sims")
    plot_data <- covid_summary2[1:nn,]
    plot_data$state <- rawdata$state
    plot_data$n <- rawdata$n
    plot_data$wdth <- plot_data$X97.5 - plot_data$X2.5
    if (!onscreen) png(output_file1, height=6*120, width=3*120)
    p <- ggplot(plot_data, aes(x=state, y=Mean)) +
        geom_linerange(mapping=aes(ymin=X2.5, ymax=X97.5)) +
        geom_linerange(mapping=aes(ymin=X25, ymax=X75), lwd=2, color="red")+
        geom_point(color="blue")+labs(y="mean prevalence", x="")+ylim(0, 1)+
        annotate("text",x=min(ordered(plot_data$state)),y=mean(plot_data$X97.5),
                 hjust=.2,label=paste("Last checked:", chckd))
    print(p+coord_flip())
    if (!onscreen) dev.off()

    ## ordered by mean
    oo<-order(plot_data$Mean)
    plot_data$stateO <- ordered(plot_data$state, levels=plot_data$state[oo])
    if(!onscreen) png(output_file2, height=6*120, width=3*120)
    pO <- ggplot(arrange(plot_data, Mean), aes(x=stateO, y=Mean))+
        geom_linerange(mapping=aes(ymin=X2.5, ymax=X97.5)) +
        geom_linerange(mapping=aes(ymin=X25, ymax=X75), lwd=2, color="red")+
        geom_point(color="blue")+ylim(0, 1)+
        labs(y="mean prevalence", x="")+
        annotate("text",x=min(plot_data$stateO),y=mean(plot_data$X97.5),
                 hjust=.2,label=paste("Last checked:", chckd))
    print(pO+coord_flip())
    if (!onscreen) dev.off()

    ## ordered by sample size
    oo <- plot_data$n
    if(!onscreen) png(output_file3, height=3*120, width=5*120)
    pO2 <- ggplot(arrange(plot_data, n), aes(x=n, y=wdth))+
        geom_point(color="blue")+ scale_x_continuous(trans = 'log10')+
        labs(y="width of 95% interval", x="number of tests")+
        annotate("text",x=quantile(plot_data$n, prob=0.95),y=max(plot_data$wdth),
                 hjust=.2,label=paste("Last checked:", date_checked))
    print(pO2)
    if (!onscreen) dev.off()
    return(list(summ=covid_summary2, pltdata=plot_data))
}



Three scenarios

In [ ]:
## best case scenario
bestcase <- covid_plot(stanfit=fitB2, rawdata=covid_daily[tmp,],
                       output_file1="prevalence1_1.png",
                       output_file2="prevalence1_2.png",
                       output_file3="prevalence1_3.png",
                       onscreen=T)

## middle case
medcase <- covid_plot(stanfit=fitB3, rawdata=covid_daily[tmp,],
                      output_file1="prevalence2_1.png",
                      output_file2="prevalence2_2.png",
                      output_file3="prevalence2_3.png",
                      onscreen=T)

## worst case
wrstcase <- covid_plot(stanfit=fitB4, rawdata=covid_daily[tmp,],
                       output_file1="prevalence3_1.png",
                       output_file2="prevalence3_2.png",
                       output_file3="prevalence3_3.png",
                       onscreen=T)



Combined plots

In [ ]:
## combined plots
cmb_data <- rbind(bestcase[[2]], medcase[[2]], wrstcase[[2]])
mm <- dim(bestcase[[2]])[1]
cmb_data$scenarios <- c(rep("best", mm), rep("expected", mm), rep("worst", mm))
png("allscenarios.png", height=6*120, width=9*120)
pp <- ggplot(data=cmb_data, aes(x=stateO, y=Mean)) +
        geom_linerange(mapping=aes(ymin=X2.5, ymax=X97.5)) +
        geom_linerange(mapping=aes(ymin=X25, ymax=X75), lwd=2, color="red")+
        geom_point(color="blue")+labs(y="mean prevalence", x="")
print(pp+facet_wrap(.~scenarios)+coord_flip())
dev.off()
